In [ ]:
import pandas as pd
import datetime
import numpy as np

from plot import remove_begin_end, read_file, get_files, get_weather_data
list_of_files = get_files()

In [ ]:
data = [remove_begin_end(read_file(filename)) for filename in list_of_files]
data = pd.concat(data).sort_index()
data.index.min(), data.index.max()

In [ ]:
df_weather = get_weather_data().sort_index()
df_weather = df_weather.loc[
        (df_weather.index > data.index.min()) & (df_weather.index < data.index.max())
    ]

In [ ]:
diff_index = np.diff(data.index)
mask_index_step = diff_index > pd.Timedelta(30, 'minutes')
mask_index_step = np.append(mask_index_step, [False])
data[mask_index_step]

In [ ]:
data_raw = [read_file(filename) for filename in list_of_files]
data_raw = pd.concat(data_raw).sort_index()

In [ ]:
len(data), len(data_raw)

In [ ]:
data_smooth = data.rolling(window='5min').mean().round(2)
data_smooth = data_smooth[data_smooth['temperature'].diff() != 0]
len(data_smooth)

In [ ]:
#notes = pd.read_csv('data/notes.txt', header=None, sep=' ', parse_dates=[[0, 1]])
#notes = notes.set_index("0_1")
#notes

In [ ]:
from scipy.optimize import curve_fit
import numpy as np

def myexp(t, a, b, c):
    return a * np.exp(-t / b) + c

def mylin(t, a, b):
    return a + b * t


def fit_span(series, function, p0):
    start = series.index.min()
    x = series.index - start
    x = x.seconds
    y = series.values
    yinf = series.iloc[-1]
    
    popt, pcov = curve_fit(function, x, y, p0=p0)
    
    return popt, pcov

In [ ]:
# riscaldamento 0.25 K/h - 0.44 K/h
# exp b = 1.44625327e+04, 1.04721363e+04

In [ ]:
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go


fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(
    go.Scatter(x=data_raw.index, y=data_raw.temperature, name="temperature raw"),
    secondary_y=False,
)

fig.add_trace(
    go.Scatter(x=data_smooth.index, y=data_smooth.temperature, name="temperature", line_color='#EF553B'),
    secondary_y=False,
)
fig.add_trace(
    go.Scatter(x=data_smooth.index, y=data_smooth.humidity, name="humidity", line_color='#202AB6', visible='legendonly'),
    secondary_y=True,
    
)

fig.add_trace(
    go.Scatter(x=df_weather.index, y=df_weather['temperature_2m'], visible='legendonly', name='ext temperature'))


# Set y-axes titles
fig.update_yaxes(title_text="temperature (°C)", ticklen=10, linewidth=1, linecolor="white", tickcolor="white", ticks='inside', secondary_y=False, mirror=True, showline=True, minor=dict(ticklen=6, tickcolor="white", ticks='inside', showgrid=True))
fig.update_yaxes(title_text="humidity (%)", ticklen=10, tickcolor="white", ticks='inside', secondary_y=True, showgrid=False, zeroline=False, minor=dict(ticklen=6, tickcolor="white", ticks='inside', showgrid=False))
fig.update_xaxes(ticklen=10, linewidth=1, linecolor="white", tickcolor="white", ticks='inside', showline=True, mirror='all')


fit_timeranges = [
#    (datetime.datetime(2022, 12, 6, 15, 4), datetime.datetime(2022, 12, 6, 22, 0), myexp),
#    (datetime.datetime(2022, 12, 7, 8, 17), datetime.datetime(2022, 12, 7, 16, 57), myexp), 
#    (datetime.datetime(2022, 12, 11, 9, 43), datetime.datetime(2022, 12, 11, 11, 47), mylin),
#    (datetime.datetime(2022, 12, 12, 9, 30), datetime.datetime(2022, 12, 12, 12, 40), mylin)
]
fit_results = []

for start, stop, function in fit_timeranges:
    start = pd.Timestamp(start).tz_localize(LOCAL_TIMEZONE)
    stop = pd.Timestamp(stop).tz_localize(LOCAL_TIMEZONE)
    data_to_fit_mask = (data.index > start) & (data.index < stop)
    data_to_fit = data[data_to_fit_mask]['temperature']

    if function == myexp:
        p0 = (data_to_fit.iloc[0] - data_to_fit.iloc[-1], 15000, data_to_fit.iloc[-1])
    else:
        p0 = (20, 1E-5)
    popt, pcov = fit_span(data_to_fit, function, p0=p0)
    print(popt)

    xspan_seconds = np.linspace(0, (data_to_fit.index.max() - data_to_fit.index.min()).seconds, 100)
    yfit = function(xspan_seconds, *popt)
    xspan_time = data_to_fit.index.min() + pd.TimedeltaIndex(xspan_seconds, 's')
    fig.add_trace(
        go.Scatter(x=xspan_time, y=yfit, showlegend=False, hoverinfo='none'),
    secondary_y=False
)
    
    fit_results.append(popt)
    
mask_working_hours = (data.index.time < datetime.time(19, 0, 0))
mask_working_hours &= (data.index.time > datetime.time(7, 0, 0))
mask_working_hours &= data.index.weekday < 5

data_working = data[mask_working_hours]

df = data_working.groupby(data_working.index.date)['temperature'].agg([min, max, 'first', 'last'])
    
fig.add_trace(go.Candlestick(x=pd.DatetimeIndex(df.index)+ datetime.timedelta(hours=13),
                                     close=df['max'],
                                     open=df['min'],
                                     high=df['max'],
                                     low=df['min'],
                             name='Min/Max in working days 7-19'
                ))

for v in pd.date_range(data.index.min(), data.index.max(), freq='D', normalize=True)[1:]:
    fig.add_vline(v, line_dash='dot', line_color='#777')

fig.update_layout(xaxis_rangeslider_visible=False)

fig.update_layout(legend=dict(
    orientation="h",
    yanchor="bottom",
    y=1.02,
    xanchor="left",
    x=0.
    ),
    template='plotly_dark'
)
    

fig.show()

In [ ]:
#fig = make_subplots(specs=[[{"secondary_y": True}]])
fig = go.FigureWidget()
fig.layout.hovermode = 'closest'
fig.layout.hoverdistance = -1 #ensures no "gaps" for selecting sparse data


default_linewidth = 2
highlighted_linewidth_delta = 1
for k, v in data_working.groupby(data_working.index.date)['temperature']:
    x = [datetime.datetime.combine(datetime.date.today(), x) for x in v.index.time]
    trace = go.Scatter(x=x, y=v.values - v.mean(), name=str(k), line_color='rgba(255, 0, 0, 0.3)', mode='lines', line={'width': default_linewidth})
    fig.add_trace(trace)


fig.update_layout(template='plotly_dark', showlegend=False)
fig.update_xaxes(ticklen=10, linewidth=1, linecolor="white", tickcolor="white", ticks='inside', showline=True, mirror='all')
fig.update_yaxes(ticklen=10, linewidth=1, linecolor="white", tickcolor="white", ticks='inside', mirror='all', title_text="temperature - mean temperature (°C)", showline=True, minor=dict(ticklen=6, tickcolor="white", ticks='inside', showgrid=True))

# our custom event handler
def update_trace(trace, points, selector):
    # this list stores the points which were clicked on
    # in all but one trace they are empty
    if len(points.point_inds) == 0:
        return
        
    for i,_ in enumerate(fig.data):
        fig.data[i]['line']['width'] = default_linewidth + highlighted_linewidth_delta * (i == points.trace_index)
        fig.data[i]['line']['color'] = 'white' if (i == points.trace_index) else 'rgba(255, 0, 0, 0.3)'



# we need to add the on_click event to each trace separately       
for i in range( len(fig.data) ):
    fig.data[i].on_hover(update_trace)

fig.write_html('test.html')
fig

In [ ]:
fig = make_subplots(specs=[[{'type': 'polar'}]])
for is_working_day, df in data.groupby(data.index.weekday < 5):
    name = {True: 'business', False: 'weekend'}[is_working_day]
    fig.add_trace(go.Scatterpolar(r=df.temperature,
                                  name=name, opacity=0.5,
                                  theta=(df.index - df.index.normalize()).seconds / (24 * 3600) * 360))
                             
fig.update_layout(
    width=750, height=750,
    template='plotly_dark',
    polar = dict(
        radialaxis_range=[data.temperature.min(), 26],
        angularaxis=dict(
            tickvals = [hr / 24 * 360 for hr in range(24)],
            ticktext = [str(x) for x in range(24)]
        )
    ))

fig.show()

In [ ]:
weather_data.keys()

In [ ]:
fig = make_subplots(specs=[[{'type': 'polar'}]])
fig.add_trace(go.Scatter(x=weather_data['hourly']['time'], y=weather_data['hourly']['temperature_2m']))

